In [3]:
# Now lets download the trained model from the last notebook using joblib
import joblib

# Load the trained model from last notebook
model = joblib.load('/content/drive/MyDrive/Colab Notebooks/XAIentists/Trained Models/xgboost_model.pkl')

In [7]:
import shap
import lime
import lime.lime_tabular
import joblib
import numpy as np
import pandas as pd
import xgboost
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [11]:
data =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XAIentists/Data/Processed/compas-scores-two-years-processed.csv')

# Drop the two target columns
X = data.drop('two_year_recid', axis=1)
y = data['two_year_recid']

#model = joblib.load('../Trained Models/xgboost_model.pkl')

model = joblib.load("/content/drive/MyDrive/Colab Notebooks/XAIentists/Trained Models/xgboost_model.pkl", mmap_mode="r")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
#rf_model.fit(X_train, y_train)

feature_names = X_train.columns.tolist()

In [24]:
print(X_train.shape)  # Check the number of rows and columns

(5771, 19)


In [22]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
X_train_encoded = encoder.fit_transform(X_train)

# Ensure it's a NumPy array
X_train_encoded = pd.DataFrame(X_train_encoded)

In [26]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_encoded.values,
    training_labels=y_train,
    feature_names=encoder.get_feature_names_out(),
    class_names=["No Recidivism", "Recidivism"],
    mode="classification"
)
# Select a sample instance from the test set
i = 5  # Choose any index
instance = X_test.iloc[i].values

#instance = X_train.iloc[0]  # Ensure it's a valid row from your training data

# Generate explanation
exp = explainer.explain_instance(instance, model.predict_proba, num_features=10)

# Visualize explanation
exp.show_in_notebook()

/usr/local/lib/python3.11/dist-packages/lime/discretize.py:110: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ret[feature] = int(self.lambdas[feature](ret[feature]))
/usr/local/lib/python3.11/dist-packages/lime/discretize.py:110: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  ret[feature] = int(self.lambdas[feature](ret[feature]))


IndexError: index 19 is out of bounds for axis 0 with size 19

In [ ]:
# Create SHAP explainer
explainer = shap.TreeExplainer(model)  # Random Forest is tree-based

# Compute SHAP values
shap_values = explainer.shap_values(X_test)

# Summary plot (global feature importance)
shap.summary_plot(shap_values[1], X_test, feature_names=feature_names)  # Class 1 (Recidivism)

# Explanation for a single instance
shap.force_plot(explainer.expected_value[1], shap_values[1][i], X_test.iloc[i], matplotlib=True)

In [ ]:
# Initialize JavaScript visualization to understand how the model makes decisions (for each feature)
shap.initjs()
shap.force_plot(shap_values[1][0], X_test[0], feature_names=feature_names)